In [133]:
from classifiers import CoTrainingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle

## Load Models

### TF-IDF

In [134]:
with open ("tfidf.pk", 'rb') as fin:
    tfidf = pickle.load(fin)

In [135]:
tfidf

TfidfVectorizer(max_df=5, ngram_range=(1, 3), stop_words='english')

### Word2Vec CBOW

In [136]:
cbow = Word2Vec.load("cbow.model")

In [137]:
cbow

### Word2Vec SkipGram

In [138]:
skg = Word2Vec.load("skg.model")

In [139]:
skg

## Importing Data

In [140]:
df = pd.read_csv("processed_train.csv")

In [141]:
df

,Unnamed: 0,id,tweet,subtask_a,subtask_b,subtask_c
0,0,86426,"['ask', 'native', 'american', 'take']",OFF,UNT,NaN
1,1,90194,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF,TIN,IND
2,2,16820,"['amazon', 'investigating', 'chinese', 'employ...",NOT,NaN,NaN
3,3,62688,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF,UNT,NaN
4,4,43605,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT,NaN,NaN
...,...,...,...,...,...,...
13235,13235,95338,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF,TIN,IND
13236,13236,67210,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT,NaN,NaN
13237,13237,82921,"['report', 'garbage', 'dont', 'give', 'crap']",OFF,TIN,OTH
13238,13238,27429,['pussy'],OFF,UNT,NaN


### Removing Redundant Axes

In [142]:
df = df.drop(['Unnamed: 0', 'subtask_b', 'subtask_c', 'id'], axis=1)

In [143]:
df

,tweet,subtask_a
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Renaming Columns

In [144]:
df = df.rename(columns={'subtask_a': 'Offensive'})

In [145]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Converting Offensive to Numerical Value

In [146]:
def off(cls):
    if cls =='OFF':
        return 1
    return 0

In [147]:
df["Offensive"] = df["Offensive"].apply(off)

In [148]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",1
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",1
2,"['amazon', 'investigating', 'chinese', 'employ...",0
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",1
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",0
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",1
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",0
13237,"['report', 'garbage', 'dont', 'give', 'crap']",1
13238,['pussy'],1


In [149]:
df['tweet']

0                    ['ask', 'native', 'american', 'take']
1        ['go', 'home', '’', 'drunk', 'maga', 'trump', ...
2        ['amazon', 'investigating', 'chinese', 'employ...
3        ['someone', 'shouldve', 'taken', 'piece', 'shi...
4        ['obama', 'wanted', 'liberal', 'amp', 'illegal...
                               ...                        
13235    ['sometimes', 'get', 'strong', 'vibe', 'people...
13236    ['benidorm', '✅', 'creamfields', '✅', 'maga', ...
13237        ['report', 'garbage', 'dont', 'give', 'crap']
13238                                            ['pussy']
13239    ['spanishrevenge', 'v', 'justice', 'human', 'r...
Name: tweet, Length: 13240, dtype: object

## Fitting Word Models on Data

In [150]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['Offensive'], stratify=df['Offensive'], random_state=0)

In [151]:
X_train = X_train.sort_index().reset_index().drop(['index'], axis=1)

In [152]:
X_train = X_train['tweet']

In [153]:
X_train

0                   ['ask', 'native', 'american', 'take']
1       ['go', 'home', '’', 'drunk', 'maga', 'trump', ...
2       ['obama', 'wanted', 'liberal', 'amp', 'illegal...
3                                   ['liberal', 'kookoo']
4       ['literally', 'talking', 'lol', 'mass', 'shoot...
                              ...                        
9925                       ['brightest', 'light', 'tree']
9926    ['sometimes', 'get', 'strong', 'vibe', 'people...
9927    ['benidorm', '✅', 'creamfields', '✅', 'maga', ...
9928        ['report', 'garbage', 'dont', 'give', 'crap']
9929    ['spanishrevenge', 'v', 'justice', 'human', 'r...
Name: tweet, Length: 9930, dtype: object

In [154]:
X_test = X_test.sort_index().reset_index().drop(['index'], axis=1)

In [155]:
X_test = X_test['tweet']

In [156]:
X_test

0       ['amazon', 'investigating', 'chinese', 'employ...
1       ['someone', 'shouldve', 'taken', 'piece', 'shi...
2                           ['oh', 'no', 'tough', 'shit']
3       ['canada', '’', 'need', 'another', 'cuck', 'al...
4       ['da', 'fuck', 'going', 'people', 'there', 'me...
                              ...                        
3305    ['conservative', 'accepted', 'antisemitism', '...
3306    ['much', 'lonely', 'much', 'miss', 'azeez', 'd...
3307    ['😂', '😂', '😂', 'say', 'mad', 'say', 'im', 'ti...
3308    ['retweet', 'complete', 'amp', 'followed', 'pa...
3309                                            ['pussy']
Name: tweet, Length: 3310, dtype: object

### TF-IDF

In [163]:
X_train_tf = pd.DataFrame(tfidf.transform(X_train.sort_index()).todense())
print(X_train_tf)

      0       1       2       3       4       5       6       7       8       \
0        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
9925     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9926     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9927     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9928     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9929     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

      9       ...  153232  153233  1532

In [160]:
X_test_tf = pd.DataFrame(tfidf.transform(X_test.sort_index()).todense())

In [161]:
X_test_tf

,0,1,2,3,4,5,6,7,8,9,...,153232,153233,153234,153235,153236,153237,153238,153239,153240,153241
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Sentence Vectoriser

Finding the average vector for a given document

In [175]:
def sent_vect(sent, model):
    sent = eval(sent)
    vec = np.zeros(32)
    num = 0
    for w in sent:
        try:
            temp_vec = model.wv.get_vector(w)
            #print(temp_vec)
            vec = np.add(vec, temp_vec)
            num += 1
        except:
            pass    
    vec =  vec / np.sqrt(vec.dot(vec))
    #print(type(vec))
    return vec

### CBOW

In [176]:
X_train_cbow = X_train.apply(sent_vect, model=cbow).sort_index()

In [177]:
X_train_cbow

0       [0.11678230874736859, 0.04701858112529256, 0.1...
1       [0.023887964134799274, -0.04221032997250825, 0...
2       [0.023089552048990755, -0.036891745280832305, ...
3       [0.08365669444110241, 0.024382291167563585, 0....
4       [0.1692174577116111, -0.028866439854166506, 0....
                              ...                        
9925    [0.05910713569141994, 0.03033006832724931, 0.2...
9926    [0.1772960903524207, 0.06816052902725968, 0.19...
9927    [-0.07846772515660165, -0.09172202196583056, 0...
9928    [0.1720133490472096, 0.0603833914465248, 0.166...
9929    [0.029200375665037092, -0.027191909181750278, ...
Name: tweet, Length: 9930, dtype: object

In [181]:
X_test_cbow = X_test.apply(sent_vect, model=cbow).sort_index()

In [182]:
X_test_cbow

0       [-0.05144699518391731, -0.07000303002084465, 0...
1       [0.10839988236478272, -0.02007087490801532, 0....
2       [0.14215383139067395, 0.09612889083374788, 0.0...
3       [0.14834794895835923, 0.041729775171655985, 0....
4       [0.10737067888646858, 0.06788053575314568, 0.0...
                              ...                        
3305    [0.07261746754903796, 0.023324642599656067, 0....
3306    [0.11278004817370701, 0.06896479115835308, 0.1...
3307    [0.11422348815947328, -0.03982285937728821, 0....
3308    [-0.04389888644976187, -0.1105844745764699, 0....
3309    [0.06189782589780631, 0.05593079023601799, 0.1...
Name: tweet, Length: 3310, dtype: object

### SkipGram

In [179]:
X_train_skg = X_train.apply(sent_vect, model=skg).sort_index()

In [180]:
X_train_skg

0       [0.13890833028577879, 0.0602093866169931, 0.25...
1       [-0.031044015373555757, -0.06391622386404926, ...
2       [0.009941511388440752, 0.006697370287529366, 0...
3       [0.05088327548790825, 0.12449198214735306, 0.2...
4       [0.20541999370674277, -0.011201584641294629, 0...
                              ...                        
9925    [0.0820706311814619, 0.05471222033757831, 0.22...
9926    [0.10886295323000372, 0.06637772834474372, 0.1...
9927    [-0.04250946890486125, -0.005367833791252949, ...
9928    [0.18586203624193126, 0.07148610685130281, 0.2...
9929    [0.02059650501444647, -0.010558274514424206, 0...
Name: tweet, Length: 9930, dtype: object

In [184]:
X_test_skg = X_test.apply(sent_vect, model=skg).sort_index()

In [185]:
X_test_skg

0       [-0.020249622204438412, -0.01119598407878388, ...
1       [0.1714072956388892, -0.03793757609649255, 0.1...
2       [0.1370963143540046, 0.09248017762336874, 0.13...
3       [0.10742154726732796, 0.04232172938983719, 0.2...
4       [0.08625167256161481, 0.07987306727597052, 0.1...
                              ...                        
3305    [0.08095425708329977, 0.10200269965214329, 0.1...
3306    [0.08540180817933385, 0.029416859234483287, 0....
3307    [0.1665732372459549, -0.09866697941966471, 0.1...
3308    [0.014044533632225758, -0.12783930366160623, 0...
3309    [0.10757330260668012, 0.09901826002879813, 0.1...
Name: tweet, Length: 3310, dtype: object

## Co - Training

### SVM vs SVM

In [173]:
svm2 = CoTrainingClassifier(SVC())

In [178]:
svm2.fit(X_train_tf, X_train_cbow, y_train)

ValueError: setting an array element with a sequence.